# Data : 영어와 프랑스어 문장

**Raw** : 약 15만 개의 문장

**Extract** : 5만 개
- eng : 50,000 (`input`)
- fr : 50,000 (`output`)

**Preprocessing**
- 소문자 변환, 구두점 제거


- 프랑스어(`output`) 문장에 `start`와 `end` 토큰을 추가


- 누적 비율 80%에 해당하는 단어들을 추출


- 누적 비율 80% 단어에 해당하지 않는 단어는 `unk`단어로 변환


- unique 단어 list 생성
    - `all_eng_words`, `all_french_words`
    
    
- input, target word 생성
    - `input_words` = sorted(list(all_eng_words))
    - `target_words` = sorted(list(all_french_words))
    - `num_encoder_tokens` = len(all_eng_words) : 385
    - `num_decoder_tokens` = len(all_french_words) : 358


- 단어에 해당하는 index 생성
    - `input_token_index` : input_words에 대한 index
    - `target_token_index` : target_words에 대한 index
    
    
- 모든 문장의 길이가 동일하도록 최대길이를 추출
    - `fr_max_length` : 17
    - `eng_max_length` : 8
    
    
- input, output 데이터셋 생성
    - `encoder_input_data`(전체 문장, 문장의 단어길이) : input 문장의 단어들을 순서대로 input_token_index에 해당하는 값을 할당(50000, 17)    
        
    - `decoder_input_data`(전체 문장, 문장의 단어길이) : target 문장의 단어들을 순서대로 target_token_index에 해당하는 값을 할당(50000, 17)
    
    - `decoder_target_data`(전체 문장, 문장의 단어길이, target_token_index 크기) :     
        target 문장의 단어들을 순서대로(start 토큰은 제외한 index로) 각 단어에 해당하는 target_token_index의 값을 index로 하여 1를 할당(50000, 17, 359)
        > 문장의 길이를 초과하는 단어의 index들에는 `end`토큰의 index(89)에 해당하는 index에 1을 할당한다.

# Model 1 : Traditional many to many architecture


이 아키텍처에서 우리는 각 입력 단어를 128 차원 벡터로 embed 할 것이다. 그 결과 출력 벡터의 shape은 (batch_size, 128, 17) 이다.

이 버전에서 입력 데이터가 17개의 time step을 가지고 있고 출력 데이터 또한 17개의 time step을 가진다.

*여기서 17은 전체 문장들 중에 최대 단어의 개수를 나타낸다.

> ```python
model = Sequential() 
model.add(Embedding(len(input_words)+1, 128, input_length=fr_max_length, mask_zero=True))
model.add((Bidirectional(LSTM(256, return_sequences = True))))
model.add((LSTM(256, return_sequences=True)))
model.add((Dense(len(target_token_index)+1, activation='softmax')))
model.summary()
```

- acc : 약 19%

# Model 2 : Many to hidden to many architecture

이전 아키텍처의 단점 중 하나는 입력에 최대 8개의 time step가 있다는 것을 알면서도 입력의 time step 수를 인위적으로 17개로 늘려야 한다는 것이었다.

이 아키텍처에서는 입력의 마지막 단계에서 `hidden state` 값을 추출하는 모델을 구축해 보자.

또한 `hidden state` 값을 17회 복제한다(출력에는 17개의 시간 단계가 있으므로). 

복제된 hidden time step을 Dense layer을 통과하여 output에서 높은 가능성을 가지는 class를 최종적으로 추출한다. 

**입력에 8 time step, 출력에 17 time step이 되도록 입력 및 출력 데이터셋 생성**


> ```python
model2 = Sequential()
model2.add(Embedding(len(input_words)+1, 128, input_length=eng_max_length, mask_zero=True))
model2.add((Bidirectional(LSTM(256))))
model2.add(RepeatVector(fr_max_length))
model2.add((LSTM(256, return_sequences=True)))
model2.add((Dense(len(target_token_index)+1, activation='softmax')))
```
**RepeatVector layer**는 bidirectional layer의 output을 17(fr_max_length)번 반복한다.

- acc : 약 19%

- 모든 input time step의 정보가 마지막 hidden layer 값에만 저장될 때 상당한 양의 정보가 손실되는 경향이 있기 때문에 The preceding is expected,
- 또한, 어떤 time step에서 forgot되야할 것들에 대한 상당한 정보를 포함하는 **cell state**를 사용하지 않는다.

# Model 3 : Encoder decoder architecture for machine translation

이전 섹션에서 정의한 아키텍처에는 두 가지 잠재적인 향상 가능성이 있다.
1. 번역을 생성하는 동안 **cell state**에 존재하는 정보를 활용한다.

2. 앞서 번역한 단어를 다음 단어의 예측에 입력으로 활용한다.(**Teacher Forcing**)
    > 이전 time step의 실제 값을 입력으로 주면서 현재 time step을 생성함으로써 네트워크를 보다 빠르고, 실질적으로 더 정확하게 튜닝할 수 있다.
    
기계 번역 시스템을 구축하기 위해 채택할 인코더 디코더 아키텍처는 다음과 같다.

---
- 두 개의 decoder 데이터셋
    - `encoder_input_data`와 결합된 **decoder_input_data**가 입력이고 **decoder_target_data**가 출력이다.

  - **decoder_input_data**는 `start` 단어로 시작된다.


- 우리가 decoder에서 첫번째 단어를 예측할 때, 우리는 단어들의 input set을 벡터로 변환하고, 그 다음 input으로서 `start`를 decoder 모델에 통과시킨다. 예상되는 output은 output에서 `start`뒤의 첫 번째 단어이다.


- 비슷한 방식으로 output의 실제 첫 번째 단어가 input으로 두 번째 단어를 예측한다.


- 두 번째 단어를 예측하면서 출력의 실제 첫 단어가 입력인 유사한 방식으로 진행한다.


- 이 전략을 바탕으로 모델의 정확성을 계산한다.
---
encoder network의 중간 layer를 추출하고 input(encoder input data와 decoder input data)으로 여러 데이터셋을 전달하므로 API 함수를 사용한다.

- acc : 44%
---
실제 상황에서는 `decoder_input_data`에 접근할 수 없으므로 test dataset의 정확도를 계산할 때 `decoder_input_data`를 사용해서는 안 된다것을 주의해야한다.

따라서, 이전 time step에서 예측된 단어를 현재 time step의 decoder input 단어로 사용해야 한다.

- acc : 46%

> 이전 방법들보다 정확도가 상당히 향상되었지만 `source language`의 시작 부분에 있는 단어가 `target language`에서도 시작될 가능성이 매우 높다는 직관을 여전히 가지지 않고 있다. 즉, 단어의 alignment(정렬)이 고려되지 않았다.

# Model 4  : Encoder decoder architecture with attention for machine translation

Model3 에서는 target의 이전 time step의 실제 단어가 모델의 input으로 사용되는 **teacher forcing**을 사용함으로써 번역의 정확성을 높일 수 있다는 것을 배웠다.

이번에는 각 time step에서 encoder와 decoder가 얼마나 유사한지에 기반하여 input encoder에 가중치를 할당한다.

이 방법을 사용하면, 특정 단어가 decoder의 time step에 따라 encoder의 hidden vector에서 높은 가중치를 가질 수 있다.

---
attention 매카니즘과 함께 encoder decoder 아키텍처를 구축하는 방법을 살펴보자.

1. Build the encoder


2. Build the decoder
    - decoder에서 hidden layer 값만 추출한다.
    
    
3. Build the attention mechanism
    - attention mechanism는 각 time step에서 encoder hidden vector와 decoder hidden vector가 얼마나 유사한지를 기반으로 한다.
    - 이 유사성(softmax를 사용하여 모든 가능한 input time step에서 합이 1이 되는 가중치를 구한다.)에 기반하여 encoder vector에 가중치를 할당한다.
    - encoder decoder vector를 activation 및 dense layer에 전달하여 벡터간의 내적(유사성 척도 - 코사인 유사도)을 구하기 전에 비선형성을 달성한다.
    
    
4. Combine the decoder and weighted encoder vector


5. Connect the combination of decoder and weighted encoded vector to output layer:


6. Compile and fit the model


7. Calculate the accuracy
    - acc : 52%
    
---
지금까지의 모델들이 꽤 괜찮은 번역을 한다는 것을 알 수 있지만, 잠재적인 개선 영역이 몇 가지 있다.
- 단어 유사성에 대한 설명
    - 제나, 제이와 같은 단어들은 상당히 비슷하기 때문에 정확도를 떨어뜨리고 있음에도 불구하고 많은 패널티를 줘서는 안 된다


- `unk` 수 감소
    - 데이터셋의 차원을 줄이기 위해 `unk` 단어수를 줄였다.
    - 더 큰 코퍼스를 수집하고 산업적인 규모의 환경을 가진다면 높은 차수의 데이터를 작업할 수 있다.

https://wikidocs.net/22893